# SQLi World Model

In [1]:
import os, datetime
import numpy as np
import joblib
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer,text_to_word_sequence

## Collect HTML files

Load the HTML files

In [2]:
files = []

for f in os.listdir(os.path.join(os.getcwd(),'html')):
    fd = open(os.path.join(os.getcwd(),'html',f),'r')
    content = fd.read()
    files.append(content)
    fd.close()

Show a sample file

In [3]:
files[0]

'\n<!-- saved from url=(0042)http://jabba.hackingarena.no:806/index.php -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></head><body>Login as admin for the flag!\n<br>some content, some content\n<form action="http://jabba.hackingarena.no:806/index.php" method="post">\n<table width="100">\n<tbody><tr><td>Name:</td>\n<td><input type="text" name="username" value=""></td></tr>\n<tr><td>Password:</td>\n<td><input type="text" name="passwd" value=""></td></tr>\n\n<tr><td><input type="submit" value="Submit"></td></tr>\n</tbody></table>\n</form>\n\n</body></html>\n'

Add a mock file as a concatentation of a previous one

In [4]:
files.append(files[0]*10)

## Tokenize the files

Define the tokenization parameters

In [5]:
lower = False
filters = '"#$%&()*+,.;?@[\\]^_`{|}~\t\n'

Show an example of tokenization

In [6]:
text_to_word_sequence(files[0],lower=lower, filters=filters)

['<!--',
 'saved',
 'from',
 'url=',
 '0042',
 'http://jabba',
 'hackingarena',
 'no:806/index',
 'php',
 '-->',
 '<html><head><meta',
 'http-equiv=',
 'Content-Type',
 'content=',
 'text/html',
 'charset=UTF-8',
 '></head><body>Login',
 'as',
 'admin',
 'for',
 'the',
 'flag!',
 '<br>some',
 'content',
 'some',
 'content',
 '<form',
 'action=',
 'http://jabba',
 'hackingarena',
 'no:806/index',
 'php',
 'method=',
 'post',
 '>',
 '<table',
 'width=',
 '100',
 '>',
 '<tbody><tr><td>Name:</td>',
 '<td><input',
 'type=',
 'text',
 'name=',
 'username',
 'value=',
 '></td></tr>',
 '<tr><td>Password:</td>',
 '<td><input',
 'type=',
 'text',
 'name=',
 'passwd',
 'value=',
 '></td></tr>',
 '<tr><td><input',
 'type=',
 'submit',
 'value=',
 'Submit',
 '></td></tr>',
 '</tbody></table>',
 '</form>',
 '</body></html>']

Learn a tokenization dictionary

In [7]:
tknizer = Tokenizer(lower=lower, filters=filters)
tknizer.fit_on_texts(files)

Convert the pages into token sequences

In [8]:
tokens_list = tknizer.texts_to_sequences(files)

## Padding and trimming

Define the padding

In [9]:
tokens_per_file = 200
padding = 'NOP '

Add the padding value to the word index

In [10]:
padding_integer = len(tknizer.word_index)+1
tknizer.word_index[padding]= padding_integer

Pad short sequences and trim long sequences

In [11]:
for i in range(len(tokens_list)):
    l = len(tokens_list[i])
    if l<200:
        tokens_list[i] = tokens_list[i] + [padding_integer]*(tokens_per_file-l)
    elif l>200:
        tokens_list[i] = tokens_list[i][0:200]

## Converting into array

Flat the list and reshape it into an array

In [12]:
flat_tokens_list = [item for sublist in tokens_list for item in sublist]
tokens_array = np.array(flat_tokens_list).reshape(len(tokens_list),tokens_per_file)

Verify shape

In [13]:
tokens_array.shape

(5, 200)

## Defining VAE

Instantiating a VAE module built following [https://keras.io/examples/generative/vae/](https://keras.io/examples/generative/vae/).

*Sampling layer with reparametrization trick* $z_i\sim\mathcal{N}(\mu_\phi, \sigma_\phi)$

In [14]:
class Sampling(layers.Layer):
    
    def call(self,inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch,dim))
        return z_mean + tf.exp(.5 * z_log_var) * epsilon

*Encoder* $Q_\phi(z \vert x)$

In [15]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(200,))

x = layers.Dense(100,activation='relu')(encoder_inputs)
x = layers.Dense(25,activation='relu')(x)
z_mean = layers.Dense(latent_dim, name='z_mean')(x)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
z = Sampling()([z_mean,z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean,z_log_var,z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 100)          20100       input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 25)           2525        dense[0][0]                      
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 2)            52          dense_1[0][0]                    
____________________________________________________________________________________________

*Decoder* $P_\theta(x\vert z)$

In [16]:
latent_inputs = keras.Input(shape=(latent_dim,))

x = layers.Dense(25,activation='relu')(latent_inputs)
x = layers.Dense(100,activation='relu')(x)
decoder_outputs = layers.Dense(200,activation='relu')(x)

decoder = keras.Model(latent_inputs, decoder_outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_2 (Dense)              (None, 25)                75        
_________________________________________________________________
dense_3 (Dense)              (None, 100)               2600      
_________________________________________________________________
dense_4 (Dense)              (None, 200)               20200     
Total params: 22,875
Trainable params: 22,875
Non-trainable params: 0
_________________________________________________________________


*VAE with KL+reconstruction loss*

In [17]:
class VAE(keras.Model):
    def __init__(self,encoder,decoder,**kwargs):
        super(VAE,self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name='total_loss')
        self.reconstruction_loss_tracker = keras.metrics.Mean(name='reconstruction_loss')
        self.kl_loss_tracker = keras.metrics.Mean(name='kl_loss')
    
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def train_step(self,data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)           
            reconstruction = self.decoder(z)
            print(reconstruction)
            reconstruction_loss = keras.losses.mean_squared_error(data,reconstruction)
            #reconstruction_loss = tf.reduce_sum(keras.losses.mean_squared_error(data,reconstruction), axis=1)
            kl_loss = -.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = (tf.reduce_sum(kl_loss,axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss,self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads,self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            'loss': self.total_loss_tracker.result(),
            'reconstruction_loss': self.reconstruction_loss_tracker.result(),
            'kl_loss': self.kl_loss_tracker.result(),
        }

## Training the VAE

Instantiating and training the model

In [65]:
vae = VAE(encoder,decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(tokens_array, epochs=2000, batch_size=3)

Epoch 1/2000
Tensor("decoder/dense_4/Relu:0", shape=(None, 200), dtype=float32)
Tensor("decoder/dense_4/Relu:0", shape=(None, 200), dtype=float32)
2/2 [==============================] - 0s 13ms/step - loss: 524.9074 - reconstruction_loss: 649.6558 - kl_loss: 58.2097
Epoch 2/2000
2/2 [==============================] - 0s 9ms/step - loss: 901.7167 - reconstruction_loss: 914.3387 - kl_loss: 108.8038
Epoch 3/2000
2/2 [==============================] - 0s 6ms/step - loss: 1190.2814 - reconstruction_loss: 1058.1490 - kl_loss: 186.0988
Epoch 4/2000
2/2 [==============================] - 0s 7ms/step - loss: 596.4730 - reconstruction_loss: 449.3421 - kl_loss: 143.5385
Epoch 5/2000
2/2 [==============================] - 0s 17ms/step - loss: 1047.3309 - reconstruction_loss: 949.5701 - kl_loss: 152.1594
Epoch 6/2000
2/2 [==============================] - 0s 10ms/step - loss: 537.1570 - reconstruction_loss: 488.0240 - kl_loss: 76.7229
Epoch 7/2000
2/2 [==============================] - 0s 16ms/step

2/2 [==============================] - 0s 3ms/step - loss: 248.9017 - reconstruction_loss: 216.6268 - kl_loss: 16.3182
Epoch 62/2000
2/2 [==============================] - 0s 3ms/step - loss: 234.6942 - reconstruction_loss: 225.1532 - kl_loss: 27.1665
Epoch 63/2000
2/2 [==============================] - 0s 5ms/step - loss: 251.2819 - reconstruction_loss: 220.1250 - kl_loss: 16.6969
Epoch 64/2000
2/2 [==============================] - 0s 10ms/step - loss: 235.2006 - reconstruction_loss: 200.6546 - kl_loss: 21.0987
Epoch 65/2000
2/2 [==============================] - 0s 5ms/step - loss: 280.7875 - reconstruction_loss: 230.8357 - kl_loss: 30.2674
Epoch 66/2000
2/2 [==============================] - 0s 3ms/step - loss: 305.6975 - reconstruction_loss: 262.9886 - kl_loss: 20.3505
Epoch 67/2000
2/2 [==============================] - 0s 5ms/step - loss: 246.3401 - reconstruction_loss: 218.2246 - kl_loss: 38.3714
Epoch 68/2000
2/2 [==============================] - 0s 4ms/step - loss: 228.0114 

2/2 [==============================] - 0s 6ms/step - loss: 196.5870 - reconstruction_loss: 180.7120 - kl_loss: 24.5061
Epoch 123/2000
2/2 [==============================] - 0s 4ms/step - loss: 196.0645 - reconstruction_loss: 180.8271 - kl_loss: 21.6353
Epoch 124/2000
2/2 [==============================] - 0s 14ms/step - loss: 215.0423 - reconstruction_loss: 184.2534 - kl_loss: 19.3755
Epoch 125/2000
2/2 [==============================] - 0s 6ms/step - loss: 199.8284 - reconstruction_loss: 181.3748 - kl_loss: 23.7410
Epoch 126/2000
2/2 [==============================] - 0s 3ms/step - loss: 205.0770 - reconstruction_loss: 190.9297 - kl_loss: 17.8215
Epoch 127/2000
2/2 [==============================] - 0s 4ms/step - loss: 198.7498 - reconstruction_loss: 183.3990 - kl_loss: 23.1407
Epoch 128/2000
2/2 [==============================] - 0s 4ms/step - loss: 218.1779 - reconstruction_loss: 188.6730 - kl_loss: 21.0748
Epoch 129/2000
2/2 [==============================] - 0s 7ms/step - loss: 24

2/2 [==============================] - 0s 11ms/step - loss: 290.3568 - reconstruction_loss: 237.3537 - kl_loss: 27.8415
Epoch 184/2000
2/2 [==============================] - 0s 6ms/step - loss: 343.4688 - reconstruction_loss: 295.2673 - kl_loss: 12.3958
Epoch 185/2000
2/2 [==============================] - 0s 10ms/step - loss: 254.1072 - reconstruction_loss: 232.4787 - kl_loss: 30.0979
Epoch 186/2000
2/2 [==============================] - 0s 8ms/step - loss: 227.8959 - reconstruction_loss: 232.0284 - kl_loss: 14.4998
Epoch 187/2000
2/2 [==============================] - 0s 30ms/step - loss: 241.2249 - reconstruction_loss: 214.6825 - kl_loss: 20.6297
Epoch 188/2000
2/2 [==============================] - 0s 9ms/step - loss: 382.1843 - reconstruction_loss: 303.1638 - kl_loss: 39.2771
Epoch 189/2000
2/2 [==============================] - 0s 10ms/step - loss: 524.8669 - reconstruction_loss: 450.9977 - kl_loss: 17.9866
Epoch 190/2000
2/2 [==============================] - 0s 9ms/step - loss:

2/2 [==============================] - 0s 5ms/step - loss: 185.1750 - reconstruction_loss: 173.8508 - kl_loss: 19.2492
Epoch 245/2000
2/2 [==============================] - 0s 10ms/step - loss: 184.6172 - reconstruction_loss: 173.2535 - kl_loss: 20.7455
Epoch 246/2000
2/2 [==============================] - 0s 8ms/step - loss: 204.2612 - reconstruction_loss: 175.8364 - kl_loss: 21.0494
Epoch 247/2000
2/2 [==============================] - 0s 6ms/step - loss: 191.7041 - reconstruction_loss: 182.1535 - kl_loss: 16.7126
Epoch 248/2000
2/2 [==============================] - 0s 12ms/step - loss: 202.5669 - reconstruction_loss: 174.6571 - kl_loss: 21.3197
Epoch 249/2000
2/2 [==============================] - 0s 9ms/step - loss: 183.3776 - reconstruction_loss: 170.7562 - kl_loss: 18.5982
Epoch 250/2000
2/2 [==============================] - 0s 11ms/step - loss: 184.2799 - reconstruction_loss: 171.7710 - kl_loss: 16.8881
Epoch 251/2000
2/2 [==============================] - 0s 6ms/step - loss: 

...

2/2 [==============================] - 0s 4ms/step - loss: 180.9465 - reconstruction_loss: 165.2493 - kl_loss: 17.9174
Epoch 1701/2000
2/2 [==============================] - 0s 3ms/step - loss: 153.6427 - reconstruction_loss: 137.4388 - kl_loss: 18.3053
Epoch 1702/2000
2/2 [==============================] - 0s 4ms/step - loss: 153.9806 - reconstruction_loss: 138.2826 - kl_loss: 20.6138
Epoch 1703/2000
2/2 [==============================] - 0s 4ms/step - loss: 146.7013 - reconstruction_loss: 130.5230 - kl_loss: 20.8614
Epoch 1704/2000
2/2 [==============================] - 0s 8ms/step - loss: 151.0805 - reconstruction_loss: 135.8779 - kl_loss: 22.0255
Epoch 1705/2000
2/2 [==============================] - 0s 9ms/step - loss: 142.0773 - reconstruction_loss: 122.9853 - kl_loss: 21.8217
Epoch 1706/2000
2/2 [==============================] - 0s 5ms/step - loss: 159.7157 - reconstruction_loss: 138.2000 - kl_loss: 19.3843
Epoch 1707/2000
2/2 [==============================] - 0s 4ms/step - lo

2/2 [==============================] - 0s 5ms/step - loss: 117.8248 - reconstruction_loss: 113.0386 - kl_loss: 7.7056
Epoch 1761/2000
2/2 [==============================] - ETA: 0s - loss: 141.7575 - reconstruction_loss: 133.4500 - kl_loss: 8.307 - 0s 5ms/step - loss: 128.2108 - reconstruction_loss: 113.5764 - kl_loss: 7.8611
Epoch 1762/2000
2/2 [==============================] - 0s 5ms/step - loss: 115.1720 - reconstruction_loss: 112.6732 - kl_loss: 7.9659
Epoch 1763/2000
2/2 [==============================] - 0s 5ms/step - loss: 124.9527 - reconstruction_loss: 110.6180 - kl_loss: 8.6576
Epoch 1764/2000
2/2 [==============================] - 0s 4ms/step - loss: 134.9057 - reconstruction_loss: 119.9985 - kl_loss: 7.2083
Epoch 1765/2000
2/2 [==============================] - 0s 3ms/step - loss: 120.6402 - reconstruction_loss: 116.2312 - kl_loss: 7.9317
Epoch 1766/2000
2/2 [==============================] - 0s 6ms/step - loss: 126.6108 - reconstruction_loss: 112.0068 - kl_loss: 8.4796
Ep

2/2 [==============================] - 0s 8ms/step - loss: 114.7420 - reconstruction_loss: 111.0947 - kl_loss: 6.9546
Epoch 1821/2000
2/2 [==============================] - 0s 5ms/step - loss: 115.4178 - reconstruction_loss: 110.6505 - kl_loss: 6.3960
Epoch 1822/2000
2/2 [==============================] - 0s 4ms/step - loss: 112.0285 - reconstruction_loss: 112.8595 - kl_loss: 6.4057
Epoch 1823/2000
2/2 [==============================] - 0s 7ms/step - loss: 114.6966 - reconstruction_loss: 115.0551 - kl_loss: 5.9673
Epoch 1824/2000
2/2 [==============================] - 0s 5ms/step - loss: 132.2668 - reconstruction_loss: 129.3440 - kl_loss: 6.4811
Epoch 1825/2000
2/2 [==============================] - 0s 3ms/step - loss: 168.9451 - reconstruction_loss: 152.6821 - kl_loss: 8.0294
Epoch 1826/2000
2/2 [==============================] - 0s 5ms/step - loss: 272.3319 - reconstruction_loss: 235.1322 - kl_loss: 24.9255
Epoch 1827/2000
2/2 [==============================] - 0s 6ms/step - loss: 18

2/2 [==============================] - 0s 5ms/step - loss: 123.9963 - reconstruction_loss: 110.6928 - kl_loss: 6.2908
Epoch 1881/2000
2/2 [==============================] - 0s 10ms/step - loss: 112.5681 - reconstruction_loss: 111.0542 - kl_loss: 6.4723
Epoch 1882/2000
2/2 [==============================] - 0s 7ms/step - loss: 115.9560 - reconstruction_loss: 112.2142 - kl_loss: 6.4602
Epoch 1883/2000
2/2 [==============================] - 0s 5ms/step - loss: 112.4514 - reconstruction_loss: 112.3806 - kl_loss: 6.5934
Epoch 1884/2000
2/2 [==============================] - 0s 6ms/step - loss: 122.1786 - reconstruction_loss: 112.2461 - kl_loss: 6.2601
Epoch 1885/2000
2/2 [==============================] - 0s 4ms/step - loss: 117.1556 - reconstruction_loss: 113.5088 - kl_loss: 6.5969
Epoch 1886/2000
2/2 [==============================] - 0s 16ms/step - loss: 110.5175 - reconstruction_loss: 110.4380 - kl_loss: 6.6712
Epoch 1887/2000
2/2 [==============================] - 0s 3ms/step - loss: 1

2/2 [==============================] - 0s 5ms/step - loss: 125.4291 - reconstruction_loss: 109.9480 - kl_loss: 8.2131
Epoch 1942/2000
2/2 [==============================] - 0s 7ms/step - loss: 117.4386 - reconstruction_loss: 112.5609 - kl_loss: 8.3461
Epoch 1943/2000
2/2 [==============================] - 0s 10ms/step - loss: 116.3947 - reconstruction_loss: 111.0166 - kl_loss: 8.2835
Epoch 1944/2000
2/2 [==============================] - 0s 5ms/step - loss: 123.4286 - reconstruction_loss: 109.9942 - kl_loss: 8.2046
Epoch 1945/2000
2/2 [==============================] - 0s 5ms/step - loss: 122.9525 - reconstruction_loss: 111.0787 - kl_loss: 7.6186
Epoch 1946/2000
2/2 [==============================] - 0s 6ms/step - loss: 123.8303 - reconstruction_loss: 110.4740 - kl_loss: 7.5203
Epoch 1947/2000
2/2 [==============================] - 0s 8ms/step - loss: 114.4815 - reconstruction_loss: 111.2422 - kl_loss: 7.9201
Epoch 1948/2000
2/2 [==============================] - 0s 5ms/step - loss: 12

## Checking the VAE

Printing the encoding of the first three data points (showing $\mu_\phi, \log\sigma_\phi, z$)

In [66]:
vae.encoder(tokens_array[0:3])

[<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 2.3456044 , -1.4626523 ],
        [-2.4570012 ,  2.0801358 ],
        [-0.23118107,  4.569852  ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-4.406884 , -4.8628297],
        [-1.7679614, -2.3994868],
        [-1.6476262, -2.249241 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 2.2758093, -1.4036723],
        [-2.2363672,  2.0302722],
        [-0.3137067,  4.6862164]], dtype=float32)>]

Computing the encoding and decoding of the first datapoint

In [67]:
xtilde = vae.decoder(vae.encoder(tokens_array[0:1]))

Rounding the computed representation to integers

In [68]:
xtilde_int = np.round(xtilde)

Printing the original and the reconstructed representation.

In [69]:
len(tokens_array[0])

200

In [70]:
tokens_array[0]

array([13, 14, 15, 16, 47,  4,  5, 10, 11, 17, 18, 19, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29, 48, 12, 49, 12, 30, 31,  4,  5, 10, 11, 32, 33,
        6, 34, 35, 36,  6, 37,  7,  1,  8,  9, 38,  2,  3, 39,  7,  1,  8,
        9, 40,  2,  3, 41,  1, 42,  2, 43,  3, 44, 45, 46, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55])

In [71]:
xtilde_int.shape

(1, 200)

In [72]:
xtilde_int

array([[13., 14., 15., 16., 47.,  0.,  5.,  9., 11.,  0., 18., 19., 20.,
        21., 22., 23., 24., 25., 26., 27., 28., 29.,  0., 21., 46., 15.,
        24., 25.,  5.,  8., 14., 14., 27., 34., 12., 34., 30., 35.,  6.,
        28.,  7.,  0., 13.,  8., 32.,  0.,  3., 30.,  7.,  0., 13.,  8.,
        32.,  0.,  0.,  0.,  0., 40.,  0., 42.,  8., 44., 46., 47., 55.,
        54., 54., 55., 55., 55., 55., 55., 55., 55., 55.,  0., 55., 55.,
        55., 55., 55., 55., 55., 55., 55., 55., 55.,  0., 55., 55., 55.,
        55., 55., 55., 55., 55., 55., 55., 55., 55., 55., 55., 55., 55.,
        55., 55., 55., 55., 55., 55., 55., 55., 55., 55., 55., 55.,  0.,
        55., 55., 55., 55., 54., 55., 55., 55., 55., 55., 55., 55., 55.,
        55., 55., 55., 55.,  0., 55., 55., 54., 55., 55., 55., 55., 55.,
         0., 55., 55., 55., 55., 55., 55., 54.,  0., 55., 55., 55., 55.,
        55., 55., 55., 55., 55., 55., 55., 54., 55., 55., 55., 55., 55.,
        55., 55., 55., 54., 55., 55., 55., 55., 55.

In [73]:
tknizer.sequences_to_texts(tokens_array[0:1])

['<!-- saved from url= 0042 http://jabba hackingarena no:806/index php --> <html><head><meta http-equiv= Content-Type content= text/html charset=UTF-8 ></head><body>Login as admin for the flag! <br>some content some content <form action= http://jabba hackingarena no:806/index php method= post > <table width= 100 > <tbody><tr><td>Name:</td> <td><input type= text name= username value= ></td></tr> <tr><td>Password:</td> <td><input type= text name= passwd value= ></td></tr> <tr><td><input type= submit value= Submit ></td></tr> </tbody></table> </form> </body></html>']

In [74]:
tknizer.sequences_to_texts(xtilde_int)

['<!-- saved from url= 0042 hackingarena name= php <html><head><meta http-equiv= Content-Type content= text/html charset=UTF-8 ></head><body>Login as admin for the flag! content= </body></html> from ></head><body>Login as hackingarena text saved saved for <table content <table <form width= > the <td><input <!-- text method= ></td></tr> <form <td><input <!-- text method= passwd submit text </tbody></table> </body></html> 0042 login login login login login login login']

In [75]:
np.mean(xtilde_int==tokens_array[0])

0.735

## Saving/loading the VAE

Saving the model weights

In [29]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S%f")
vae.save_weights('ignore_vae_W_'+timestamp)

Loading the model weights

In [30]:
vae1 = VAE(encoder,decoder)
vae1.load_weights('ignore_vae_W_20210228141045773142')

Checking the weights

In [31]:
vae.decoder(vae.encoder(tokens_array[0:1])) == vae1.decoder(vae.encoder(tokens_array[0:1]))

<tf.Tensor: shape=(1, 200), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  

## Define a mock server

We import and instantiate a new server.

In [32]:
import SQLiServer

In [33]:
env = SQLiServer.mockSQLienv(files)

In [34]:
env.reset()

('\n<!-- saved from url=(0033)http://jabba.hackingarena.no:806/ -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></head><body>Login as admin for the flag!\n\n<form action="http://jabba.hackingarena.no:806/index.php" method="post">\n<table width="100">\n<tbody><tr><td>Name:</td>\n<td><input type="text" name="username" value=""></td></tr>\n<tr><td>Password:</td>\n<td><input type="text" name="passwd" value=""></td></tr>\n\n<tr><td><input type="submit" value="Submit"></td></tr>\n</tbody></table>\n</form>\n\n</body></html>',
 0,
 False,
 'env reset')

We check the current solution.

In [43]:
S0,S1 = env._get_solution()

We take a couple of random steps (corresponding to *wrong request*)

In [37]:
env.step(2)

('\n<!-- saved from url=(0042)http://jabba.hackingarena.no:806/index.php -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></head><body>Login as admin for the flag!\n<br>incorrect login\n<form action="http://jabba.hackingarena.no:806/index.php" method="post">\n<table width="100">\n<tbody><tr><td>Name:</td>\n<td><input type="text" name="username" value=""></td></tr>\n<tr><td>Password:</td>\n<td><input type="text" name="passwd" value=""></td></tr>\n\n<tr><td><input type="submit" value="Submit"></td></tr>\n</tbody></table>\n</form>\n\n</body></html>',
 -1,
 False,
 'wrong request')

In [38]:
env.step(S1)

('\n<!-- saved from url=(0042)http://jabba.hackingarena.no:806/index.php -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></head><body>Login as admin for the flag!\n<br>incorrect login\n<form action="http://jabba.hackingarena.no:806/index.php" method="post">\n<table width="100">\n<tbody><tr><td>Name:</td>\n<td><input type="text" name="username" value=""></td></tr>\n<tr><td>Password:</td>\n<td><input type="text" name="passwd" value=""></td></tr>\n\n<tr><td><input type="submit" value="Submit"></td></tr>\n</tbody></table>\n</form>\n\n</body></html>',
 -1,
 False,
 'wrong request')

We discover the right escape character.

In [39]:
env.step(S0)

('\n<!-- saved from url=(0042)http://jabba.hackingarena.no:806/index.php -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></head><body>Login as admin for the flag!\n<br>some content, some content\n<form action="http://jabba.hackingarena.no:806/index.php" method="post">\n<table width="100">\n<tbody><tr><td>Name:</td>\n<td><input type="text" name="username" value=""></td></tr>\n<tr><td>Password:</td>\n<td><input type="text" name="passwd" value=""></td></tr>\n\n<tr><td><input type="submit" value="Submit"></td></tr>\n</tbody></table>\n</form>\n\n</body></html>\n',
 -1,
 False,
 'correct escape')

We take another couple of random steps.

In [40]:
env.step(2)

('\n<!-- saved from url=(0042)http://jabba.hackingarena.no:806/index.php -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></head><body>Login as admin for the flag!\n<br>incorrect login\n<form action="http://jabba.hackingarena.no:806/index.php" method="post">\n<table width="100">\n<tbody><tr><td>Name:</td>\n<td><input type="text" name="username" value=""></td></tr>\n<tr><td>Password:</td>\n<td><input type="text" name="passwd" value=""></td></tr>\n\n<tr><td><input type="submit" value="Submit"></td></tr>\n</tbody></table>\n</form>\n\n</body></html>',
 -1,
 False,
 'wrong request')

In [41]:
env.step(S0)

('\n<!-- saved from url=(0042)http://jabba.hackingarena.no:806/index.php -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></head><body>Login as admin for the flag!\n<br>incorrect login\n<form action="http://jabba.hackingarena.no:806/index.php" method="post">\n<table width="100">\n<tbody><tr><td>Name:</td>\n<td><input type="text" name="username" value=""></td></tr>\n<tr><td>Password:</td>\n<td><input type="text" name="passwd" value=""></td></tr>\n\n<tr><td><input type="submit" value="Submit"></td></tr>\n</tbody></table>\n</form>\n\n</body></html>',
 -1,
 False,
 'wrong request')

We find the flag

In [42]:
env.step(S1)

('\n<!-- saved from url=(0042)http://jabba.hackingarena.no:806/index.php -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></head><body>Login as admin for the flag!\n<br>flag\n<form action="http://jabba.hackingarena.no:806/index.php" method="post">\n<table width="100">\n<tbody><tr><td>Name:</td>\n<td><input type="text" name="username" value=""></td></tr>\n<tr><td>Password:</td>\n<td><input type="text" name="passwd" value=""></td></tr>\n\n<tr><td><input type="submit" value="Submit"></td></tr>\n</tbody></table>\n</form>\n\n</body></html>\n',
 100,
 True,
 'flag')